<a href="https://colab.research.google.com/github/jigsawcoder/Automated-Machine-Learning-Tools/blob/master/MLBox_framingham.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install mlbox

In [ ]:
import mlbox
from mlbox.preprocessing import *
from mlbox.optimisation import *
from mlbox.prediction import *
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
#data_path = "/content/drive/My Drive/framingham.csv"
#data = pd.read_csv(data_path)
#train,test = train_test_split(data,test_size=0.3)
#test = test.drop('TenYearCHD', axis=1)


In [ ]:
#train.to_csv("/content/drive/My Drive/framingham_train.csv")
#test.to_csv("/content/drive/My Drive/framingham_test.csv")

In [ ]:
train_path = "/content/drive/My Drive/framingham_train.csv"
test_path = "/content/drive/My Drive/framingham_test.csv"


In [ ]:
paths = [train_path, test_path]
target_name = 'TenYearCHD'

In [ ]:
data = Reader(sep=",").train_test_split(paths,target_name)



reading csv : framingham_train.csv ...
cleaning data ...
CPU time: 0.14127802848815918 seconds

reading csv : framingham_test.csv ...
cleaning data ...
CPU time: 0.07819914817810059 seconds

> Number of common features : 15

gathering and crunching for train and test datasets ...
reindexing for train and test datasets ...
dropping training duplicates ...
dropping constant variables on training set ...

> Number of categorical features: 0
> Number of numerical features: 15
> Number of training samples : 3392
> Number of test samples : 848

> Top sparse features (% missing values on train set):
glucose       9.0
education     2.4
BPMeds        1.4
totChol       1.0
cigsPerDay    0.5
dtype: float64

> Task : classification
0.0    2886
1.0     506
Name: TenYearCHD, dtype: int64

encoding target ...


In [ ]:
# Removing the Drifting Variables
dft=Drift_thresholder()
data=dft.fit_transform(data)


computing drifts ...
CPU time: 0.314176082611084 seconds

> Top 10 drifts

('sysBP', 0.05346375489498034)
('currentSmoker', 0.04716981132075482)
('totChol', 0.043237139551441706)
('education', 0.04287835973656118)
('age', 0.038918570888216486)
('cigsPerDay', 0.03689104774830887)
('glucose', 0.028433995639017384)
('diaBP', 0.021234758810964705)
('male', 0.02063679245283012)
('prevalentHyp', 0.018573113207547287)

> Deleted variables : []
> Drift coefficients dumped into directory : save


## The heavy lifting : optimizing

In [ ]:
# Setting up optimiser object
opt = Optimiser(scoring='accuracy')

/usr/local/lib/python3.6/dist-packages/mlbox/optimisation/optimiser.py:74: UserWarning: Optimiser will save all your fitted models into directory 'save/joblib'. Please clear it regularly.
  +str(self.to_path)+"/joblib'. Please clear it regularly.")


In [ ]:
warnings.filterwarnings('ignore', category=DeprecationWarning)
score1 = opt.evaluate(None, data)

No parameters set. Default configuration is tested

##################################################### testing hyper-parameters... #####################################################

>>> NA ENCODER :{'numerical_strategy': 'mean', 'categorical_strategy': '<NULL>'}

>>> CA ENCODER :{'strategy': 'label_encoding'}

>>> ESTIMATOR :{'strategy': 'LightGBM', 'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.8, 'importance_type': 'split', 'learning_rate': 0.05, 'max_depth': -1, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 500, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'random_state': None, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'silent': True, 'subsample': 0.9, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'nthread': -1, 'seed': 0}


MEAN SCORE : accuracy = 0.8387382075471698
VARIANCE : 0.006191037735849059 (fold 1 = 0.8325471698113207, fold 2 = 0.8449292452830188)
CPU time: 1.2176883220672607 seconds



In [ ]:

# Lets create space of multiple configurations:
space = {
        'ne__numerical_strategy':{"search":"choice",
                                 "space":[0, "mean"]},
        'ce__strategy':{"search":"choice",
                        "space":["label_encoding", "random_projection", "entity_embedding"]}, 
        'fs__threshold':{"search":"uniform",
                        "space":[0.001, 0.2]}, 
        'est__strategy':{"search":"choice", 
                         "space":["RandomForest", "ExtraTrees", "LightGBM"]},
        'est__max_depth':{"search":"choice", 
                          "space":[8, 9, 10, 11, 12, 13]}
        }




In [ ]:
params = opt.optimise(space, data)       # It returns the best hyperparameters


##################################################### testing hyper-parameters... #####################################################
>>> NA ENCODER :{'numerical_strategy': 0, 'categorical_strategy': '<NULL>'}
>>> CA ENCODER :{'strategy': 'label_encoding'}
>>> FEATURE SELECTOR :{'strategy': 'l1', 'threshold': 0.1632295841350735}
>>> ESTIMATOR :{'strategy': 'RandomForest', 'max_depth': 8, 'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 400, 'n_jobs': -1, 'oob_score': False, 'random_state': 0, 'verbose': 0, 'warm_start': False}
MEAN SCORE : accuracy = 0.8520047169811321
VARIANCE : 0.0 (fold 1 = 0.8520047169811321, fold 2 = 0.8520047169811321)
CPU time: 2.8306312561035156 seconds
##################################################### tes

In [ ]:
score2 = opt.evaluate(params, data)


##################################################### testing hyper-parameters... #####################################################

>>> NA ENCODER :{'numerical_strategy': 0, 'categorical_strategy': '<NULL>'}

>>> CA ENCODER :{'strategy': 'label_encoding'}

>>> FEATURE SELECTOR :{'strategy': 'l1', 'threshold': 0.1632295841350735}

>>> ESTIMATOR :{'strategy': 'RandomForest', 'max_depth': 8, 'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 400, 'n_jobs': -1, 'oob_score': False, 'random_state': 0, 'verbose': 0, 'warm_start': False}


MEAN SCORE : accuracy = 0.8520047169811321
VARIANCE : 0.0 (fold 1 = 0.8520047169811321, fold 2 = 0.8520047169811321)
CPU time: 2.661893367767334 seconds



In [ ]:
pred = Predictor()
pred.fit_predict(params, data)


fitting the pipeline ...


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


CPU time: 1.5684559345245361 seconds

predicting ...


/usr/local/lib/python3.6/dist-packages/mlbox/prediction/predictor.py:392: UserWarning: Unable to get feature importances !
  warnings.warn("Unable to get feature importances !")


CPU time: 0.2109205722808838 seconds

> Overview on predictions : 

        0.0       1.0  TenYearCHD_predicted
0  0.917199  0.082801                     0
1  0.922477  0.077523                     0
2  0.760978  0.239022                     0
3  0.924507  0.075493                     0
4  0.720018  0.279982                     0
5  0.948270  0.051730                     0
6  0.950748  0.049252                     0
7  0.891860  0.108140                     0
8  0.955389  0.044611                     0
9  0.954781  0.045219                     0

dumping predictions into directory : save ...


ImportError: ignored